# Fine-Tuning BERT for Text Classification

# Dataset Preparation
## Load the CSV

In [14]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [15]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import pandas as pd # Import the pandas library

file_path = '/content/drive/MyDrive/BertDataset/comments.csv'
df = pd.read_csv(file_path)

## check

In [17]:
print(f"Total rows in CSV: {len(df)}")
print(df.head())

Total rows in CSV: 97554
                                                text     label
0  ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...  Positive
1  أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...  Positive
2  هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...  Positive
3  خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...  Positive
4  ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...  Positive


## Preprocess Text

In [18]:
# Clean the data (remove special symbols and handle NaN)
def clean_text(text):
    if isinstance(text, str):  # Ensure it's a string before processing
        # Remove the special characters
        text = text.replace('?', '').replace('&', '').replace('#', '').replace('$', '').replace('%', '').replace('@', '').replace('*', '').replace('^', '')
        return text.lower()  # Convert to lowercase
    return ''

In [19]:
# Apply the cleaning function to the text data
df['text'] = df['text'].apply(clean_text)

# Drop rows with NaN values
df = df.dropna(subset=['text', 'label'])

In [20]:
# Shuffle the dataset
df = df.sample(frac=1).reset_index(drop=True)

# Check if data looks correct now
print(df.head())

                                                text     label
0  حسن الجندي كاتب ذكي جدا ومبدع جدا اتخذ مسار مخ...  Positive
1  جيد فقط لا غير . يقرأ لمرة يتيمة فقط ومن ثم يل...     Mixed
2  حتي تعرفو الفارق بين عبد الناصر وغيره من يريد ...  Positive
3  رحلة عمل . بهو الفندق جميل. المناشف جدا قديمة ...  Negative
4  مقبول. مواقف السيارات وموظفيهم.سهولة الخروج من...     Mixed


##Split Data (70% Train, 30% Test

In [21]:
# Split the dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

In [22]:
test_text = "هل هذا مثال؟ #اختبار"
cleaned = re.sub(r'[?&#$%@*^]', '', test_text)
print(f"Before: {test_text}\nAfter: {cleaned}")

Before: هل هذا مثال؟ #اختبار
After: هل هذا مثال؟ اختبار


# BERT Model Setup

## Choose Model

In [23]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Tokenization

##Determine max_len

In [26]:
# Tokenize the text data
def tokenize_data(text):
    # Set a fixed max_length for all sequences
    max_len = 128  # You might need to adjust this based on your data
    return tokenizer(
        text,
        padding='max_length',  # Pad to max_length
        truncation=True,       # Truncate if longer than max_length
        max_length=max_len,     # Set the maximum sequence length
        return_tensors='pt'    # Return PyTorch tensors directly
    )

train_encodings = train_df['text'].apply(tokenize_data)
test_encodings = test_df['text'].apply(tokenize_data)

In [28]:
# Convert the tokenized data and labels into torch tensors
train_input_ids = torch.cat([item['input_ids'] for item in train_encodings])
test_input_ids = torch.cat([item['input_ids'] for item in test_encodings])

train_attention_mask = torch.cat([item['attention_mask'] for item in train_encodings])
test_attention_mask = torch.cat([item['attention_mask'] for item in test_encodings])

# Assuming train_labels and test_labels are NumPy arrays and contain the correct label values
train_labels = torch.tensor(train_labels, dtype=torch.long)  # or torch.int64
test_labels = torch.tensor(test_labels, dtype=torch.long)  # or torch.int64

# Training Configuration
## Batch Size Selection

In [29]:
# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Train Setup

In [31]:
!pip install transformers --upgrade

In [ ]:
import os

# Disable W&B
os.environ["WANDB_DISABLED"] = "true"

# Continue with your training script
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',           # output directory
    num_train_epochs=1,               # number of training epochs
    per_device_train_batch_size=8,    # batch size for training
    per_device_eval_batch_size=8,     # batch size for evaluation
    gradient_accumulation_steps=8,    # gradient accumulation
    logging_dir='./logs',             # directory for storing logs
    evaluation_strategy="epoch",      # evaluate each epoch
    save_strategy="epoch",           # save model after each epoch
    load_best_model_at_end=True,      # load best model at the end of training
)

trainer = Trainer(
    model=model,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: Paste an API key from your profile and hit enter:

#  Evaluation

In [ ]:
# Evaluate the model
predictions, labels, _ = trainer.predict(torch.utils.data.TensorDataset(test_input_ids, test_attention_mask, test_labels))

# Get the predicted labels
predicted_labels = np.argmax(predictions, axis=1)

## Confusion Matrix

In [ ]:
# Plot the confusion matrix
cm = confusion_matrix(test_labels, predicted_labels)
plt.figure(figsize=(6, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
classes = ['Positive', 'Negative', 'Mix']
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.show()

# Test on Custom Sentences

In [ ]:
custom_sentences = [
    "الفيلم كان رائعاً!",                   # Positive
    "الخدمة مقبولة لكن بطيئة",              # Mixed
    "أسوأ تجربة في حياتي"                   # Negative
]

cleaned_sentences = [clean_text(s) for s in custom_sentences]
encoded_inputs = tokenizer(
    cleaned_sentences,
    truncation=True,
    padding='max_length',
    max_length=max_len,
    return_tensors='pt'
).to(device)

with torch.no_grad():
    outputs = model(**encoded_inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)

reverse_label_map = {v: k for k, v in label_map.items()}
for i, sentence in enumerate(custom_sentences):
    print(f"Sentence: {sentence}")
    print(f"Predicted sentiment: {reverse_label_map[predictions[i].item()]}")
    print("-" * 50)

##